# Common Data Format Conversion
Every of our datasources keep their metadata in different formats and have different sets of parameters. In this notebook, we'll join them all into a proper .csv file for the future data processing and learning.

### Constants and imports

In [7]:
import pandas as pd
import json
import os

METADATA_DIRECTORY = 'metadata'
CMA_METADATA_FILE = os.path.join(METADATA_DIRECTORY, 'cma.json')
MET_METADATA_FILE = os.path.join(METADATA_DIRECTORY, 'met.json')
NGA_METADATA_FILE = os.path.join(METADATA_DIRECTORY, 'nga.json')
RESULT_METADATA_FILE = os.path.join(METADATA_DIRECTORY, 'metadata_joined.csv')

### Preparing output dataframe

In [12]:
result = pd.DataFrame()

### Loading CMA data

In [13]:
if os.path.exists(CMA_METADATA_FILE):
    with open(CMA_METADATA_FILE, 'r') as f:
        cma_metadata = json.load(f)

In [14]:
if cma_metadata is not None:
    rows = []
    for art_id in cma_metadata:
        row = {}
        metadata = cma_metadata[art_id]
        row['id'] = f'cma_{art_id}'
        row['title'] = metadata['title']
        row['date'] = metadata['creation_date']
        row['begin_date'] = None
        row['end_date'] = None
        row['culture'] = metadata['culture']
        row['technique'] = metadata['technique']
        row['type'] = metadata['type']
        row['department'] = metadata['department']
        row['collection'] = metadata['collection']
        row['path'] = metadata['path']
        rows.append(row)
        
    df_cma = pd.DataFrame(rows)
    result = pd.concat([result, df_cma], ignore_index=True)

In [15]:
result.describe()

,id,title,date,culture,technique,type,department,collection,path
count,29745,29745,28507,29745,29470,29745,29745,29745,29745
unique,29745,21530,4656,4105,5976,61,19,161,29745
top,cma_122190,Page from Tales of a Parrot (Tuti-nama): text ...,1800s,"France, 19th century",etching,Print,Prints,PR - Etching,dataset/raw/cma_122190.jpg
freq,1,335,839,2807,1978,7808,7670,2870,1


### Loading MET data

In [17]:
if os.path.exists(MET_METADATA_FILE):
    with open(MET_METADATA_FILE, 'r') as f:
        met_metadata = json.load(f)

In [21]:
if met_metadata is not None:
    rows = []
    for art_id in met_metadata:
        row = {}
        metadata = met_metadata[art_id]
        if 'path' not in metadata:
            continue
        row['id'] = f'met_{art_id}'
        row['title'] = metadata['objectName']
        row['date'] = metadata['objectDate']
        row['begin_date'] = metadata['objectBeginDate']
        row['end_date'] = metadata['objectEndDate']
        row['culture'] = f"{metadata['culture']}, {metadata['period']}"
        row['technique'] = metadata['medium']
        row['type'] = metadata['classification']
        row['department'] = metadata['department']
        row['collection'] = None
        row['path'] = metadata['path']
        rows.append(row)
        
    df_met = pd.DataFrame(rows)
    result = pd.concat([result, df_met], ignore_index=True)

In [24]:
result.describe()

,begin_date,end_date
count,1904.000000,1904.000000
mean,1076.473739,1209.426471
std,1275.867259,1042.845564
min,-7000.000000,-5000.000000
25%,800.000000,999.750000
50%,1700.000000,1733.500000
75%,1814.250000,1860.250000
max,1935.000000,1999.000000


### Loading NGA data

### Saving the .csv file